In [1]:
import json

In [2]:
with open("autodiag_report.json", "r") as f:
    d = json.load(f)

In [3]:
series = d['data']['sensors']

In [4]:
series_list = []
for k, v in series.items():
    series_list.append((k, v[0][0], v))
sorted_series = sorted(series_list, key=lambda x: x[1]) # sorts by first timestamp
min_t = sorted_series[0][1]

min_t

1675716436.1571245

In [5]:
for ind,(i, j, k) in enumerate(sorted_series):
    j = j - min_t
    k = [[kk[0] - min_t, kk[1]] for kk in k]
    sorted_series[ind] = (i, j, k)

sorted_series

[('state_pre_extend_move',
  0.0,
  [[0.0,
    {'vac': -35.3960911136108,
     'touch': 5.234863281249999,
     'finger': 0.2548828125}],
   [0.006159782409667969,
    {'vac': -35.3960911136108,
     'touch': 5.531494140624999,
     'finger': 0.2548828125}],
   [0.006492137908935547,
    {'vac': -35.83548931383577,
     'touch': 5.828124999999999,
     'finger': 0.2548828125}]]),
 ('state_extend',
  0.2306511402130127,
  [[0.2306511402130127,
    {'vac': -98.01033464566929,
     'touch': 2.015869140625,
     'finger': 1.3623046875}],
   [0.2348482608795166,
    {'vac': -98.01033464566929, 'touch': 2.3125, 'finger': 1.43701171875}],
   [0.250978946685791,
    {'vac': -97.90048509561304,
     'touch': 2.0048828125,
     'finger': 1.56005859375}]]),
 ('state_wait_fingers_settled',
  0.5282332897186279,
  [[0.5282332897186279,
    {'vac': -98.01033464566929, 'touch': 2.13671875, 'finger': 1.845703125}],
   [0.6285545825958252,
    {'vac': -98.01033464566929,
     'touch': 2.11474609375,
  

In [6]:
# get y_0 and y_1 per state
def get_min_max(sensor, series):
    minv = min([min([x[1][sensor] for x in k[2]]) for k in series])
    maxv = max([max([x[1][sensor] for x in k[2]]) for k in series])
    return minv, maxv


# These will be y0 and y1 for the rectangles depending on the sensor
min_vac, max_vac = get_min_max('vac', sorted_series)
min_touch, max_touch = get_min_max('touch', sorted_series)
min_fing, max_fing = get_min_max('finger', sorted_series)
print(f"(Vacuum) Min: {min_vac} | Max: {max_vac}")
print(f"(Finger) Min: {min_fing} | Max: {max_fing}")
print(f"(Touch) Min: {min_touch} | Max: {max_touch}") 

(Vacuum) Min: -98.01033464566929 | Max: -29.354365860517436
(Finger) Min: 0.2548828125 | Max: 1.845703125
(Touch) Min: 0.36791992187499994 | Max: 5.828124999999999


In [7]:
# build rectangles. x0 and x1 will be the same for all sensors
rect_list = []
for ind, (state, t_0, t_series) in enumerate(sorted_series):
    rect = {
        "state": state,
        "x0": t_0,
    }
    try:
        rect["x1"] = sorted_series[ind + 1][1] # t0 of next state
    except IndexError:
        rect["x1"] = sorted_series[ind][2][-1][0] # The last timestamp
    rect_list.append(rect)

rect_list


[{'state': 'state_pre_extend_move', 'x0': 0.0, 'x1': 0.2306511402130127},
 {'state': 'state_extend', 'x0': 0.2306511402130127, 'x1': 0.5282332897186279},
 {'state': 'state_wait_fingers_settled',
  'x0': 0.5282332897186279,
  'x1': 2.5282039642333984},
 {'state': 'state_get_finger_data',
  'x0': 2.5282039642333984,
  'x1': 3.4022774696350098},
 {'state': 'state_retract', 'x0': 3.4022774696350098, 'x1': 3.899667739868164},
 {'state': 'state_close_schnoz_gap',
  'x0': 3.899667739868164,
  'x1': 4.04777455329895},
 {'state': 'state_wait_vac_start',
  'x0': 4.04777455329895,
  'x1': 6.728764057159424},
 {'state': 'state_vac_pull', 'x0': 6.728764057159424, 'x1': 6.960143804550171},
 {'state': 'state_wait_empty_fingers_settled',
  'x0': 6.960143804550171,
  'x1': 7.7150022983551025},
 {'state': 'state_exit_tray_zone',
  'x0': 7.7150022983551025,
  'x1': 7.73173713684082}]

In [12]:
import plotly.offline as pyo
import plotly.graph_objects as go
pyo.init_notebook_mode()

t = [x[0] for k in sorted_series for x in k[2]]
vac = [x[1]['vac'] for k in sorted_series for x in k[2]]
touch = [x[1]['touch'] for k in sorted_series for x in k[2]]
finger = [x[1]['finger'] for k in sorted_series for x in k[2]]


fig=go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=t,
#         y=vac,
#         name='Vacuum',
#         line=dict(width=3),
#     ),
    
# )

fig.add_trace(
    go.Scatter(
        x=t,
        y=touch,
        name='Touch',
        line=dict(width=3),
    ),
    
)

# for rect in rect_list:
#     x0 = rect['x0']
#     x1 = rect['x1']
#     y0 = min_vac
#     y1 = max_vac
#     fig.add_trace(
#         go.Scatter(
#             x=[x0,x1, x1, x0],
#             y=[y0,y0,y1,y1],
#             fill='toself',
#             showlegend=False,
#             name=rect['state'],
#             opacity=0.3,
#         )
#     )
    
for rect in rect_list:
    x0 = rect['x0']
    x1 = rect['x1']
    y0 = min_touch
    y1 = max_touch
    fig.add_trace(
        go.Scatter(
            x=[x0,x1, x1, x0],
            y=[y0,y0,y1,y1],
            fill='toself',
            showlegend=False,
            name=rect['state'],
            opacity=0.3,
        )
    )
print(fig)
fig.show()


In [10]:
import plotly.graph_objects as go
print(type(go))

t = [x[0] for k in sorted_series for x in k[2]]
vac = [x[1]['vac'] for k in sorted_series for x in k[2]]

print(t)
print(vac)

<class 'module'>
[0.0, 0.006159782409667969, 0.006492137908935547, 0.2306511402130127, 0.2348482608795166, 0.250978946685791, 0.5282332897186279, 0.6285545825958252, 0.7292888164520264, 2.5282039642333984, 2.6287426948547363, 2.7282938957214355, 3.4022774696350098, 3.4281022548675537, 3.5291218757629395, 3.899667739868164, 3.9285147190093994, 3.944502115249634, 4.04777455329895, 4.055134057998657, 4.072663068771362, 6.728764057159424, 6.828249216079712, 6.9286887645721436, 6.960143804550171, 6.9640820026397705, 6.965144157409668, 7.7150022983551025, 7.728471279144287, 7.73173713684082]
[-35.3960911136108, -35.3960911136108, -35.83548931383577, -98.01033464566929, -98.01033464566929, -97.90048509561304, -98.01033464566929, -98.01033464566929, -98.01033464566929, -98.01033464566929, -97.90048509561304, -98.01033464566929, -98.01033464566929, -97.90048509561304, -98.01033464566929, -98.01033464566929, -98.01033464566929, -98.01033464566929, -29.354365860517436, -30.013463160854894, -30.89